# Experiment 8

Experiment 8 aims to examine the stability of different control method with respect to starting position. The experiment will use the same methods as most previous ones however there will be greater examination of end number of links involved for each swarm control method.

In [1]:
## Imports

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import networkx as nx

import logging

from src.simulation import Simulation
from src.network import Network
from src.render import Render

logging.disable(logging.CRITICAL)

In [2]:
network_file_1 = "networks/Net3.inp"
network_file_2 = "networks/250701 K709vs2-Export.inp"
network_file_3 = "networks/Net6.inp"

env1 = Network(network_file_1)
env2 = Network(network_file_2)
env3 = Network(network_file_3)

g_env1 = env1.water_network_model.to_graph().to_undirected()
g_env2 = env2.water_network_model.to_graph().to_undirected()
g_env3 = env3.water_network_model.to_graph().to_undirected()

d_env1 = g_env1.degree
d_env2 = g_env2.degree
d_env3 = g_env3.degree

env1_start_pool = [node for node, degree in d_env1 if degree == 1]
env2_start_pool = [node for node, degree in d_env2 if degree == 1]
env3_start_pool = [node for node, degree in d_env3 if degree == 1]

env1_num_links = env1.graph_num_links
env2_num_links = env2.graph_num_links
env3_num_links = env3.graph_num_links

print("Start Pool - Net3: ", env1_start_pool)
print("Number of Start Nodes - Net3: ", len(env1_start_pool))
print("Number of Nodes - Net3: ", env1.graph_num_nodes)
print("Number of Links - Net3: ", env1.graph_num_links)

print("Start Pool - 250701 K709vs2-Export: ", env2_start_pool)
print("Number of Start Nodes - 250701 K709vs2-Export: ", len(env2_start_pool))
print("Number of Nodes - 250701 K709vs2-Export: ", env2.graph_num_nodes)
print("Number of Links - 250701 K709vs2-Export: ", env2.graph_num_links)

print("Start Pool - Net6: ", env3_start_pool)
print("Number of Start Nodes - Net6: ", len(env3_start_pool))
print("Number of Nodes - Net6: ", env3.graph_num_nodes)
print("Number of Links - Net6: ", env3.graph_num_links)

Start Pool - Net3:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2', '3']
Number of Start Nodes - Net3:  16
Number of Nodes - Net3:  97
Number of Links - Net3:  119
Start Pool - 250701 K709vs2-Export:  ['A2148', 'A3007', 'A3018', 'A5065', 'A5071', 'A5078', 'A5153', 'A5169', 'A5174', 'A5175', 'A5178', 'A5183', 'A5184', 'A5185', 'A5187', 'A5189', 'A5191', 'A5197', 'A5200', 'A5202', 'A5205', 'A5245', 'A5247', 'A5255', 'A5271', 'A5279', 'A5283', 'A5285', 'A5306', 'A5311', 'A5323', 'A5342', 'A5343', 'A5348', 'A5351', 'A5354', 'A5357', 'A5376', 'A5377', 'A5379', 'A5383', 'A5387', 'A5390', 'A5391', 'A5394', 'A5396', 'A5398', 'A5400', 'A5404', 'A5407', 'A5408', 'A5412', 'A5423', 'A5424', 'A5426', 'A5428', 'A5430', 'A5431', 'A5435', 'A5440', 'A5448', 'A5450', 'A5451', 'A5460', 'A5462', 'A5471', 'A5473', 'A5474', 'A5480', 'A5481', 'A5485', 'A5502', 'A5510', 'A5513', 'A5515', 'A5701', 'A5708', 'AN-1871', 'N-0457', 'N-0458', 'N-0459', 'N-0460',

In [3]:
def run_simulation_batch(env, num_agents, start_nodes, filepath, max_turns=100):
    print("Starting Simulation Batch - ", filepath)
    print("Number of Agents: ", num_agents)
    print("Start Nodes: ", start_nodes)
    print("Max Turns: ", max_turns)
    
    # Run the simulations for no swarm control
    path = f'{filepath}/NoSwarm'
    simulations_1 = []
    for node in start_nodes:
        print("Starting No Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=False, start_positions=[node], filepath=path)
        simulations_1.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_1
    
    # Run the simulations for naive swarm control
    path = f'{filepath}/NaiveSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'naive'}
    simulations_2 = []
    for node in start_nodes:
        print("Starting Naive Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_2.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_2
    
    # Run the simulations for informed mean swarm control
    path = f'{filepath}/InformedMeanSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'mean'}
    simulations_3 = []
    for node in start_nodes:
        print("Starting Informed Mean Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_3.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)    
    yield simulations_3
    
    # Run the simulations for informed median swarm control
    path = f'{filepath}/InformedMedianSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'median'}
    simulations_4 = []
    for node in start_nodes:
        print("Starting Informed Median Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_4.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_4

## Simulations

### Simulation 1 - Environment 1

In [4]:
start_nodes = env1_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-8/Env1"

print("Start Nodes: ", start_nodes)

Start Nodes:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2']


In [5]:
simulations_8_2 = run_simulation_batch(env1, num_agents, start_nodes, filepath, max_turns=max_turns)

no_swarm_env1 = next(simulations_8_2)
naive_swarm_env1 = next(simulations_8_2)
informed_mean_swarm_env1 = next(simulations_8_2)
informed_median_swarm_env1 = next(simulations_8_2)

Starting Simulation Batch -  notable-results/Experiment-8/Env1
Number of Agents:  10
Start Nodes:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2']
Max Turns:  100
Starting No Swarm Simulation from Start Node:  15
Starting No Swarm Simulation from Start Node:  35
Starting No Swarm Simulation from Start Node:  131
Starting No Swarm Simulation from Start Node:  166
Starting No Swarm Simulation from Start Node:  167
Starting No Swarm Simulation from Start Node:  203
Starting No Swarm Simulation from Start Node:  219
Starting No Swarm Simulation from Start Node:  225
Starting No Swarm Simulation from Start Node:  231
Starting No Swarm Simulation from Start Node:  243
Starting No Swarm Simulation from Start Node:  253
Starting No Swarm Simulation from Start Node:  River
Starting No Swarm Simulation from Start Node:  Lake
Starting No Swarm Simulation from Start Node:  1
Starting No Swarm Simulation from Start Node:  2
Starting Naive Swarm

### Simulation 2 - Environment 2

In [6]:
start_nodes = env2_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-8/Env2"

print("Start Nodes: ", start_nodes)

Start Nodes:  ['A2148', 'A3007', 'A3018', 'A5065', 'A5071', 'A5078', 'A5153', 'A5169', 'A5174', 'A5175', 'A5178', 'A5183', 'A5184', 'A5185', 'A5187']


In [7]:
simulations_8_2 = run_simulation_batch(env2, num_agents, start_nodes, filepath, max_turns=max_turns)

no_swarm_env2 = next(simulations_8_2)
naive_swarm_env2 = next(simulations_8_2)
informed_mean_swarm_env2 = next(simulations_8_2)
informed_median_swarm_env2 = next(simulations_8_2)

Starting Simulation Batch -  notable-results/Experiment-8/Env2
Number of Agents:  10
Start Nodes:  ['A2148', 'A3007', 'A3018', 'A5065', 'A5071', 'A5078', 'A5153', 'A5169', 'A5174', 'A5175', 'A5178', 'A5183', 'A5184', 'A5185', 'A5187']
Max Turns:  100
Starting No Swarm Simulation from Start Node:  A2148
Starting No Swarm Simulation from Start Node:  A3007
Starting No Swarm Simulation from Start Node:  A3018
Starting No Swarm Simulation from Start Node:  A5065
Starting No Swarm Simulation from Start Node:  A5071
Starting No Swarm Simulation from Start Node:  A5078
Starting No Swarm Simulation from Start Node:  A5153
Starting No Swarm Simulation from Start Node:  A5169
Starting No Swarm Simulation from Start Node:  A5174
Starting No Swarm Simulation from Start Node:  A5175
Starting No Swarm Simulation from Start Node:  A5178
Starting No Swarm Simulation from Start Node:  A5183
Starting No Swarm Simulation from Start Node:  A5184
Starting No Swarm Simulation from Start Node:  A5185
Startin

### Simulation 3 - Environment 3

In [ ]:
start_nodes = env3_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-8/Env3"

print("Start Nodes: ", start_nodes)

In [ ]:
simulations_8_3 = run_simulation_batch(env3, num_agents, start_nodes, filepath, max_turns=max_turns)

no_swarm_env3 = next(simulations_8_3)
naive_swarm_env3 = next(simulations_8_3)
informed_mean_swarm_env3 = next(simulations_8_3)
informed_median_swarm_env3 = next(simulations_8_3)

## Analysis

In [140]:
from typing import List

# Read the results from the simulations
def read_results(simulations, swarm_type) -> List[pd.DataFrame]:
    results = []
    for sim in simulations:
        start_node, path = sim
        df = pd.read_csv(f"{path}/results.csv")
        df['start_node'] = start_node
        df.start_node = start_node
        df['swarm_type'] = swarm_type
        df.swarm_type = swarm_type
        results.append(df)
    return results

def clean_dataframe(dataframe: pd.DataFrame) -> pd.Series:
    max_row = dataframe.loc[dataframe['abs_links_explored'].idxmax()]
    max_row = max_row[['abs_links_explored', 'start_node', 'swarm_type']]
    return max_row

def sim_dataframe(simulations, swarm_type):
    df = pd.DataFrame()
    results = read_results(simulations, swarm_type)
    for result in results:
        result = clean_dataframe(result)
        result = result.reset_index(drop=True)
        df = pd.concat([df, result], axis=1, ignore_index=True)
    return df

def make_dataframe(simulations, swarm_type):
    df = sim_dataframe(simulations, swarm_type)
    df = df.T
    df.columns = ['abs_links_explored', 'start_node', 'swarm_type']
    df = df.reset_index(drop=True)
    return df

def batch_make_dataframe(simulations:list, swarm_types:list):
    df = pd.DataFrame()
    for i in range(len(simulations)):
        df_ = make_dataframe(simulations[i], swarm_types[i])
        df = pd.concat([df, df_], axis=0, ignore_index=True)
    return df
      
    

In [52]:
data1 = read_results(no_swarm_env1, 'no-swarm')
data1[0]

,turn,pct_nodes_explored,pct_links_explored,node_novelty_score,link_novelty_score,abs_nodes_explored,abs_links_explored,start_node,swarm_type
0,0,1.030928,0.840336,1.0,1.0,1,1,15,no-swarm
1,1,2.061856,1.680672,1.0,1.0,2,2,15,no-swarm
2,2,3.092784,2.521008,1.0,1.0,3,3,15,no-swarm
3,3,4.123711,3.361345,1.0,1.0,4,4,15,no-swarm
4,4,5.154639,4.201681,1.0,1.0,5,5,15,no-swarm
...,...,...,...,...,...,...,...,...,...
95,95,54.639175,52.100840,0.0,0.0,53,62,15,no-swarm
96,96,54.639175,52.100840,0.0,0.0,53,62,15,no-swarm
97,97,54.639175,52.100840,0.0,0.0,53,62,15,no-swarm
98,98,54.639175,52.100840,0.0,0.0,53,62,15,no-swarm


In [91]:
res = clean_dataframe(data1[0])
res

abs_links_explored          62
start_node                  15
swarm_type            no-swarm
Name: 89, dtype: object

In [120]:
no_swarm_1 = make_dataframe(no_swarm_env1, 'no-swarm')
naive_1 = make_dataframe(naive_swarm_env1, 'naive')

In [143]:
sims = [no_swarm_env1, naive_swarm_env1, informed_mean_swarm_env1, informed_median_swarm_env1]
swarm_types = ['no-swarm', 'naive', 'informed-mean', 'informed-median']

df_all = batch_make_dataframe(sims, swarm_types)
df_all[df_all.start_node == 'Lake']

,abs_links_explored,start_node,swarm_type
12,27,Lake,no-swarm
27,38,Lake,naive
42,82,Lake,informed-mean
57,82,Lake,informed-median
